![Py4Eng](img/logo.png)

# [Problem 8](https://projecteuler.net/problem=8)
## Yoav Ram

We will review four participants solutions to a similar problem and then show an instructor solution.

## The problem

This is the [Largest product in a series](https://projecteuler.net/problem=8) problem from the [Euler project](https://projecteuler.net/). 

The four adjacent digits in the 1000-digit number that have the greatest product are 9 × 9 × 8 × 9 = 5832.

```
73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450
```
Find the thirteen adjacent digits in the 1000-digit number that have the greatest product. What is the value of this product?

In [1]:
number = """73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450""".replace('\n', '').replace(' ', '')

## Participant solution #1

The first solution uses imperative Pure Python approach, creating a slice for each substring and calculating it's product, then checking if it's larger than the previous maximum. Lots of copies are made (when slicing), products are calculated for each new slice with an inner loop.
In addition, the preparation doesn't pre-allocate the memory for the list of digits (not very dramatic here, could save ~0.05ms for full solution).

In [2]:
def run_problem_1(number, number_of_digits):
    all_digits = [] # better to pre-allocate memory?
    for c in number:
        try: # EAFP!
            code = int(c) - int("0") # int(c) is enough right?
            all_digits.append(code)
        except:
            ; # use pass, not required if we first replace all '\n' with ''
    input_size = len(all_digits)
    size_of_text = len(number)

    maxValue = 0 # lowercase with underscores
    maxIndex = 0

    for index in range(0, input_size-number_of_digits+1):
        slice = all_digits[index:index+number_of_digits] # slice is a python saved word
        current = 1
        for digit in slice:
            current *= digit
        if current>maxValue:
            maxValue = current
            maxIndex = index
        
    return maxValue

## Participant solution #2

Similar approach to the above with slightly different syntax.

Notes:
- Using `set` is cool but here it's not useful as the set is not returned. Probably good for debugging, though.
- Don't use an overall `try` block, as it can catch `ValueError`s that you didn't plan for. The code inside the `try` block should be as short as possible. Definately don't `return` from within a `try`! In this case probably should be (note the detailed error message):

```py
try:
  int(n)
except ValueError:
  print('Not a valid number: {}'.format(n))
```

In [3]:
def run_problem_2(n, num_digits):
#     n=n.replace('\n', '').replace(' ','') # already done above

    try: 
        int(n)
  
        max_product=0
        val_set=set()
 
        for i in range(0, len(n)-num_digits+1):
            val=n[i:i+num_digits]
            curr_product=int(val[0])
            for j in range (1,num_digits):
                curr_product *= int(val[j])
            if curr_product>max_product:
               max_product=curr_product;
               val_set={val};
            elif curr_product==max_product:
                val_set.add(val)
        return max_product
    except ValueError:
        print( 'Not a valid number')  

## Participant solution #3

A more efficient implementation uses a "running product" approach, dividing by the oldest element and only re-calculating product if said oldest element was zero, this making away with 90% of inner loops (on average).

- Probably better to convert everything to `int` in one place in the code, to limit "forgetting" to convert; of course this is not useful if requires an additional loop, therefore only useful with "lazy" generators, see below.
- Product calculation code is repeated - write a separate function.
- Same `try` comment as above.
- `;` not needed at end of line
- Should use `//` rather than `/` to have `int` return type rather than `float`.

In [4]:
def run_problem_3(digits, nd):
    try:
        product = 1
        for i in range(0, nd):
            val = digits[i]
            product *= int(val[0])
        #print("First product = {}".format(product))
 
        max_product = product
        for i in range(nd, len(digits)):
            val_first = int(digits[i - nd])
            val_next = int(digits[i])
           
            if val_first == 0:
                next_product = 1
                for j in range(i - nd + 1, i + 1):
                    next_product *= int(digits[j])
            else:
                next_product = product / val_first * val_next; # use //!
           
            if (next_product > max_product):
                max_product = next_product
           
            product = next_product
        #print("Maximal product = {}".format(max_product))
        return max_product
    except ValueError:
        print( 'Not a valid number') 

## Participant solution #4

A different approach, using modulo instead of converting string literals into integers. This is clever, but in effect, sometimes it's better to use the high-level [`int(x)`](https://docs.python.org/3.6/library/functions.html#int) which does the conversion using a more optimized version of the code here.
This function also finds and returns the exact substring, which somewhat reduces it's efficiency (~20%), but that depends on what we are interested in.

- No `;` needed at end of line
- No space needed after function name definition or after `while`.
- Creating a new list for every product calculation is wasteful, as all these list are of the same length.
- The helper function does more than the name suggests.

In [5]:
x = "318783123787321331318783123787321331318783123787321331318783123787321331"

In [6]:
%%timeit
n = int(x)
s = 0
while n:
    s += n%10
    n //= 10

100000 loops, best of 3: 14.7 µs per loop


In [7]:
%%timeit
s = 0
for c in x:
    s += int(c)

10000 loops, best of 3: 16 µs per loop


The most efficient approach, with shortest code, is to use `sum` with a generator (or `map`):

In [8]:
%timeit sum(int(x) for x in x)

100000 loops, best of 3: 17 µs per loop


In [9]:
def calc_digits_product (x):
    prod = 1
    l = [];
    while (x > 0):
        digit = x % 10
        l.append(digit)
        prod *= digit
        x //= 10
    return prod, l[::-1]

def run_problem_4(x, num_digits):
    x = int(x) #  to match API of other functions that expect str
    max_product = 1
    max_product_seq = []
    while (x > 0):
        prod, seq = calc_digits_product(x % (10 ** num_digits))
        if prod > max_product:
            max_product = prod
            max_product_seq = seq
        x //= 10
    return max_product#, max_product_seq # to match API of other functions that return product

## Instructor solution #1

For my solution I utilize some of the functional programming tools: [itertools](https://docs.python.org/3.6/library/itertools.html), [operator](https://docs.python.org/3.6/library/operator.html), [functools.reduce](https://docs.python.org/3.6/library/functools.html#functools.reduce); and the [deque](https://docs.python.org/3.6/library/collections.html?highlight=deque#collections.deque) which allows fast removal and insertion from both sides of the collection.

In [10]:
from itertools import islice
from collections import deque
from functools import reduce
import operator

product = lambda iterable: reduce(operator.mul, iterable, 1)

def run_problem_5(number, n):
    all_digits = (int(x) for x in number)
    n_digits = deque(islice(all_digits, n))
    max_prod = n_prod = product(n_digits)
    
    for newest in all_digits:
        oldest = n_digits.popleft() # remove oldest
        n_digits.append(newest) # add newest
        if oldest == 0:
            # recalc product
            n_prod = product(n_digits)
        else:
            # update product
            n_prod //= oldest 
            n_prod *= newest
        # update max product
        max_prod = max_prod if n_prod < max_prod else n_prod
    return max_prod

In [12]:
for func in (run_problem_1, run_problem_2, run_problem_3, run_problem_4, run_problem_5):
    assert func(number, 13) == 23514624000, func.__name__
    print(func.__name__, end=': ')
    %timeit func(number, 13)

run_problem_1: 1000 loops, best of 3: 1.22 ms per loop
run_problem_2: 100 loops, best of 3: 3.5 ms per loop
run_problem_3: 1000 loops, best of 3: 909 µs per loop
run_problem_4: 100 loops, best of 3: 5.27 ms per loop
run_problem_5: 1000 loops, best of 3: 546 µs per loop


## Instructor solution #2

This solution is also amendable to returning the actual substring at a very low cost (memory: O(`n`), time: depends how many maximums are found, worst case is O(`m`) for `m` length of full number).

In [13]:
def run_problem_6(number, n):
    all_digits = (int(x) for x in number)
    n_digits = deque(islice(all_digits, n))
    max_prod = n_prod = product(n_digits)
    max_digits = tuple(n_digits)
    
    for newest in all_digits:
        oldest = n_digits.popleft() # remove oldest
        n_digits.append(newest) # add newest
        if oldest == 0:
            # recalc product
            n_prod = product(n_digits)
        else:
            # update product
            n_prod //= oldest 
            n_prod *= newest
        # update max product
        max_prod = max_prod if n_prod < max_prod else n_prod
        max_digits = max_digits if n_prod < max_prod else tuple(n_digits)
    return max_prod, str.join('', (str(x) for x in max_digits))

In [15]:
run_problem_6(number, 13)

(23514624000, '5576689664895')

In [16]:
%timeit run_problem_6(number, 13)

1000 loops, best of 3: 578 µs per loop


## Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com) and is part of the [_Python for Engineers_](https://github.com/yoavram/Py4Eng) course.

The notebook was written using [Python](http://python.org/) 3.6.1.
Dependencies listed in [environment.yml](../environment.yml), full versions in [environment_full.yml](../environment_full.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)